In [ ]:
# Late vs On-Time Deliveries by Shipper

  # Load the query
query = """
SELECT 
    s.company_name AS shipper,
    CASE 
        WHEN o.shipped_date > o.required_date THEN 'Late'
        ELSE 'On-Time'
    END AS delivery_status,
    COUNT(*) AS delivery_count
FROM orders o
JOIN shippers s ON o.ship_via = s.shipper_id
WHERE o.shipped_date IS NOT NULL AND o.required_date IS NOT NULL
GROUP BY s.company_name, delivery_status;
"""
df = pd.read_sql(query, engine)

# Calculate percentage per shipper
df_total = df.groupby("shipper")["delivery_count"].sum().reset_index(name="total")
df = df.merge(df_total, on="shipper")
df["percentage"] = round((df["delivery_count"] / df["total"]) * 100, 2)

# Plot with red for Late and blue for On-Time
fig = px.bar(
    df,
    x="shipper", y="percentage",
    color="delivery_status",  barmode="stack", text_auto=True,
    title="Late vs On-Time Deliveries by Shipper (Percentage)",
    labels={"percentage": "Delivery %", "shipper": "Shipper"},
    color_discrete_map={"Late": "red", "On-Time": "blue"}
)

fig.update_layout(template="plotly_white", yaxis_ticksuffix="%")
fig.show()
